In [2]:
!pip install rembg==2.0.56

# Imports

In [7]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from rembg import new_session, remove

# Constants

In [4]:
BASE_PATH = '/kaggle/input/key-images/'

# Helper Functions

In [ ]:
def extract_key_pipeline(image_path):
    # Read the image
    img = cv2.imread(image_path)

    # Convert to grayscale
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Remove noise using Gaussian blur
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)

    # Detect edges using Canny edge detection
    edges = cv2.Canny(blurred_image, 30, 150)
    
    return edges

In [5]:
def load_general_use_model(model_name='isnet-general-use'):
    '''
    Parameters:
    model_name: str
    '''
    session = new_session(model_name)
    return session


def remove_bg(input_img, only_mask=False):
    '''
    Remove background for single images
    '''
    # Make session
    session_isnet_general_use = load_general_use_model()

    # Output image
    output_img = remove(input_img,
                        session=session_isnet_general_use,
                        only_mask=only_mask)
    return output_img

def plt_img(img_data, cmap='viridis'):
    '''
    cmap:
      * 'gray' (Grayscale)
      * 'binary' (Binary)
    '''
    plt.imshow(img_data, cmap=cmap)
    plt.axis('off')
    plt.show()

rembg

# Step1: Extracting Object (Key) from the image

In [ ]:
edges = extract_key_pipeline(f'{BASE_PATH}1a.jpeg')
plt_img(edges, cmap='binary')

In [ ]:
edges = extract_key_pipeline(f'{BASE_PATH}1b.jpeg')
plt_img(edges, cmap='binary')

In [ ]:
edges = extract_key_pipeline(f'{BASE_PATH}2a.jpeg')
plt_img(edges, cmap='binary')

In [ ]:
edges = extract_key_pipeline(f'{BASE_PATH}2b.jpeg')
plt_img(edges, cmap='binary')

In [ ]:
edges = extract_key_pipeline(f'{BASE_PATH}3.jpeg')
plt_img(edges)

In [ ]:
edges = extract_key_pipeline(f'{BASE_PATH}4.jpeg')
plt_img(edges)

In [ ]:
# Read the image
image = cv2.imread(f'{BASE_PATH}1a.jpeg')

# Convert to grayscale
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply bilateral filter to preserve edges and texture
filtered_image = cv2.bilateralFilter(gray_image, 9, 75, 75)

# Adaptive thresholding to create a binary image
binary_image = cv2.adaptiveThreshold(filtered_image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 4)

# Perform morphological operations to get the exterior boundary
# kernel = np.ones((3,3), np.uint8)
# boundary = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, kernel)

# Find contours
contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw contours
boundary_image = np.zeros_like(image)
output = cv2.drawContours(boundary_image, contours, -1, (255, 255, 255), 1)
plt_img(output)

In [ ]:
plt_img(binary_image)

In [ ]:
plt_img(inverted_image)